## Heart Attack Prediction Model 
## Data Extraction, Transformation, Export
## [2023 BRFSS Dataset](https://www.cdc.gov/brfss/annual_data/annual_2023.html)
---


## Author: Will Wu

## Date: 2025-03-14
---

## **Table of Contents** ##

1. [Load Libraries](#1)
2. [Converting SAS to PARQUET](#2)
3. [Data Preprocessing](#3)
    - [Reading, overviewing data](#3a)
    - [Variable Selection and Renaming](#3b)
    - [Map Numeric Values to Categories](#3c)
4. [Validity Check](#4)
3. [Data Export](#5)

---

## 1. Load Libraries <a class="anchor" id="1"></a>

In [4]:
# Import necessary packages
import numpy as np
import pandas as pd
import os

---

## 2. Converting SAS to PARQUET <a class="anchor" id="2"></a>

In [5]:
# Step 1: Read the .XPT file
df = pd.read_sas("../../data/Active/raw/LLCP2023.XPT", format='xport')

# Step 2: Specify the full path for the Parquet file, including the filename and extension
output_path = "../../data/Active/processed/LLCP2023.parquet"

# Step 3: Create the directory if it doesn't exist
os.makedirs("../../data/Active/processed", exist_ok=True)

# Step 4: Convert to Parquet
df.to_parquet(output_path, engine='pyarrow')


---

## 3. Data preprocessing  <a class="anchor" id="3"></a>

### 3.1 Reading, overviewing data <a class="anchor" id="3a"></a>

In [6]:
# Load files
heart_df = pd.read_parquet("../../data/Active/processed/LLCP2023.parquet")

In [7]:
# Check info
heart_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 433323 entries, 0 to 433322
Columns: 350 entries, _STATE to _DRNKDRV
dtypes: float64(345), object(5)
memory usage: 1.1+ GB


In [8]:
# Peek at the top 5 rows
heart_df.head().T

,0,1,2,3,4
_STATE,1.0,1.0,1.0,1.0,1.0
FMONTH,1.0,1.0,1.0,1.0,1.0
IDATE,b'03012023',b'01062023',b'03082023',b'03062023',b'01062023'
IMONTH,b'03',b'01',b'03',b'03',b'01'
IDAY,b'01',b'06',b'08',b'06',b'06'
...,...,...,...,...,...
_PNEUMO3,2.0,1.0,1.0,1.0,1.0
_AIDTST4,2.0,2.0,2.0,1.0,2.0
_RFSEAT2,1.0,1.0,1.0,1.0,1.0
_RFSEAT3,1.0,1.0,1.0,1.0,1.0


### 3.2 Variable Selection and Renaming <a class="anchor" id="3b"></a>

Extracting the list of variables that will be used for this project

In [9]:
# Generate an interpretable Variable Name
NEW_VAR_NAMES = [
    "State",
    "Sex",
    "GeneralHealth",
    "PhysicalHealthDays",
    "MentalHealthDays",
    "StressLevels",
    "LastCheckupTime",
    "PhysicalActivities",
    "HadHeartAttack",
    "HadAngina",
    "HadStroke",
    "HadAsthma",
    "HadSkinCancer",
    "HadCOPD",
    "HadDepressiveDisorder",
    "HadKidneyDisease",
    "HadArthritis",
    "HadDiabetes",
    "DeafOrHardOfHearing",
    "BlindOrVisionDifficulty",
    "DifficultyConcentrating",
    "DifficultyWalking",
    "DifficultyDressingBathing",
    "DifficultyErrands",
    "SmokerStatus",
    "ECigaretteUsage",
    "ChestScan",
    "RaceEthnicityCategory",
    "AgeCategory",
    "HeightInMeters",
    "WeightInKilograms",
    "BMI",
    "AlcoholDrinkers",
    "FluVaxLast12",
    "PneumoVaxEver",
    "TetanusLast10Tdap",
]

Load the variable description file that contains original feature name and feature descriptions.

In [10]:
# Load the varaible description, set up the column names
var_list_df = pd.read_csv(
    '../../docs/vars_list_with_descriptions_heart_attack_2023.txt', 
    sep=' - ', 
    header=None, 
    names=['Variable', 'Description']
    )

C:\Users\yehua\AppData\Local\Temp\ipykernel_34708\1018227411.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  var_list_df = pd.read_csv(


In [11]:
# Show the variable description
var_list_df

,Variable,Description
0,_STATE,State FIPS Code
1,SEXVAR,Sex of Respondent
2,GENHLTH,Would you say that in general your health is:
3,PHYSHLTH,"Now thinking about your physical health, which..."
4,MENTHLTH,"Now thinking about your mental health, which i..."
5,SDHSTRE1,"Within the last 30 days, how often have you fe..."
6,CHECKUP1,About how long has it been since you last visi...
7,EXERANY2,"During the past month, other than your regular..."
8,CVDINFR4,"(Ever told) you had a heart attack, also calle..."
9,CVDCRHD4,(Ever told) (you had) angina or coronary heart...


Select only the columns specified in the Variable Description File and filter out the other columns from the original DataFrame.

In [12]:
# Convert Variable name into numpy array
var_list = var_list_df['Variable'].to_numpy()

In [13]:
# Sanity Check
var_list

array(['_STATE', 'SEXVAR', 'GENHLTH', 'PHYSHLTH', 'MENTHLTH', 'SDHSTRE1',
       'CHECKUP1', 'EXERANY2', 'CVDINFR4', 'CVDCRHD4', 'CVDSTRK3',
       'ASTHMA3', 'CHCSCNC1', 'CHCCOPD3', 'ADDEPEV3', 'CHCKDNY2',
       'HAVARTH4', 'DIABETE4', 'DEAF', 'BLIND', 'DECIDE', 'DIFFWALK',
       'DIFFDRES', 'DIFFALON', '_SMOKER3', 'ECIGNOW2', 'LCSCTSC1',
       '_RACE', '_AGEG5YR', 'HTM4', 'WTKG3', '_BMI5', 'DRNKANY6',
       'FLUSHOT7', 'PNEUVAC4', 'TETANUS1'], dtype=object)

In [14]:
# Filter the data to only include rows within var_list
heart_df = heart_df[var_list]

Rename the columns to be more interpretable

In [15]:
# Rename the columns
heart_df.columns = NEW_VAR_NAMES

In [16]:
# Sanity Check
heart_df.head().T

,0,1,2,3,4
State,1.0,1.0,1.0,1.0,1.0
Sex,2.0,2.0,2.0,2.0,2.0
GeneralHealth,2.0,2.0,4.0,2.0,4.0
PhysicalHealthDays,88.0,88.0,6.0,2.0,88.0
MentalHealthDays,88.0,88.0,2.0,88.0,88.0
StressLevels,5.0,5.0,3.0,5.0,2.0
LastCheckupTime,2.0,2.0,1.0,3.0,1.0
PhysicalActivities,2.0,1.0,1.0,1.0,1.0
HadHeartAttack,2.0,2.0,2.0,2.0,2.0
HadAngina,2.0,2.0,2.0,2.0,2.0


### 3.3 Map Numeric Values to Categories <a class="anchor" id="3c"></a>

Map the numeric values to categories based on the descripitons of each survey question/feature.

In [17]:
# Create dictionaries so store the actual values in each variable 
STATE = {
    1: "Alabama",
    2: "Alaska",
    4: "Arizona",
    5: "Arkansas",
    6: "California",
    8: "Colorado",
    9: "Connecticut",
    10: "Delaware",
    11: "District of Columbia",
    12: "Florida",
    13: "Georgia",
    15: "Hawaii",
    16: "Idaho",
    17: "Illinois",
    18: "Indiana",
    19: "Iowa",
    20: "Kansas",
    21: "Kentucky",
    22: "Louisiana",
    23: "Maine",
    24: "Maryland",
    25: "Massachusetts",
    26: "Michigan",
    27: "Minnesota",
    28: "Mississippi",
    29: "Missouri",
    30: "Montana",
    31: "Nebraska",
    32: "Nevada",
    33: "New Hampshire",
    34: "New Jersey",
    35: "New Mexico",
    36: "New York",
    37: "North Carolina",
    38: "North Dakota",
    39: "Ohio",
    40: "Oklahoma",
    41: "Oregon",
    42: "Pennsylvania",
    44: "Rhode Island",
    45: "South Carolina",
    46: "South Dakota",
    47: "Tennessee",
    48: "Texas",
    49: "Utah",
    50: "Vermont",
    51: "Virginia",
    53: "Washington",
    54: "West Virginia",
    55: "Wisconsin",
    56: "Wyoming",
    66: "Guam",
    72: "Puerto Rico",
    78: "Virgin Islands"
}

SEX = {1: 'Male', 2: 'Female'}

GEN_HEALTH = {
    1: "Excellent",
    2: "Very good",
    3: "Good",
    4: "Fair",
    5: "Poor",
    7: "Unknown"
}

PHYS_MEN_HEALTH = {77: np.nan,
               88: 0,
               99: np.nan
                  }

LAST_CHECKUP = {
    1: "Within past year (anytime less than 12 months ago)",
    2: "Within past 2 years (1 year but less than 2 years ago)",
    3: "Within past 5 years (2 years but less than 5 years ago)",
    4: "5 or more years ago",
    7: 'Unknown',
    8: 'Never'
}

YES_NO_QUESTIONS = {1: 'Yes', 2: 'No'}

YES_NO_UNKNOWN_QUESTIONS = {1: "Yes", 2: "No", 7: "Unknown"}

DIABETES = {
    1: "Yes",
    2: "Yes, but only during pregnancy (female)",
    3: "No",
    4: "No, pre-diabetes or borderline diabetes",
    7: "Unknown"
}

SMOKER_STATUS = {
    1: "Current smoker - now smokes every day",
    2: "Current smoker - now smokes some days",
    3: "Former smoker",
    4: "Never smoked"
}

ECIGARETTES = {
    1: "Never used e-cigarettes in my entire life",
    2: "Use them every day",
    3: "Use them some days",
    4: "Not at all (right now)"
}

RACE = {
    1: "White only, Non-Hispanic",
    2: "Black only, Non-Hispanic",
    3: "Other race only, Non-Hispanic",
    4: "Asian only, Non-Hispanic",
    5: "Other race only, Non-Hispanic",
    7: "Multiracial, Non-Hispanic",
    8: "Hispanic",
    9: "Unknown"
}

AGE_CATEGORY = {
    1: "Age 18 to 24",
    2: "Age 25 to 29",
    3: "Age 30 to 34",
    4: "Age 35 to 39",
    5: "Age 40 to 44",
    6: "Age 45 to 49",
    7: "Age 50 to 54",
    8: "Age 55 to 59",
    9: "Age 60 to 64",
    10: "Age 65 to 69",
    11: "Age 70 to 74",
    12: "Age 75 to 79",
    13: "Age 80 or older"
}

TETANUS = {
    1: "Yes, received Tdap",
    2: "Yes, received tetanus shot, but not Tdap",
    3: "Yes, received tetanus shot but not sure what type",
    4: "No, did not receive any tetanus shot in the past 10 years",
    7: "Unknown"
}

STRESS = {
    1: "Always",
    2: "Usually",
    3: "Sometimes",
    4: "Rarely",
    5: "Never",
    7: "Unknown"
}



In [18]:
# Create a copy of heart_df
heart_copy_df = heart_df.copy()

In [19]:
# Map the numeric values back to the true values
heart_copy_df['State'] = heart_copy_df['State'].map(STATE)
heart_copy_df['Sex'] = heart_copy_df['Sex'].map(SEX)
heart_copy_df['GeneralHealth'] = heart_copy_df['GeneralHealth'].map(GEN_HEALTH)
heart_copy_df['PhysicalHealthDays'] = heart_copy_df['PhysicalHealthDays'].replace(PHYS_MEN_HEALTH)
heart_copy_df['MentalHealthDays'] = heart_copy_df['MentalHealthDays'].replace(PHYS_MEN_HEALTH)
heart_copy_df['StressLevels'] = heart_copy_df['StressLevels'].map(STRESS)
heart_copy_df['LastCheckupTime'] = heart_copy_df['LastCheckupTime'].map(LAST_CHECKUP)
heart_copy_df['PhysicalActivities'] = heart_copy_df['PhysicalActivities'].map(YES_NO_QUESTIONS)
heart_copy_df['HadHeartAttack'] = heart_copy_df['HadHeartAttack'].map(YES_NO_UNKNOWN_QUESTIONS)
heart_copy_df['HadAngina'] = heart_copy_df['HadAngina'].map(YES_NO_QUESTIONS)
heart_copy_df['HadStroke'] = heart_copy_df['HadStroke'].map(YES_NO_QUESTIONS)
heart_copy_df['HadAsthma'] = heart_copy_df['HadAsthma'].map(YES_NO_QUESTIONS)
heart_copy_df['HadSkinCancer'] = heart_copy_df['HadSkinCancer'].map(YES_NO_UNKNOWN_QUESTIONS)
heart_copy_df['HadCOPD'] = heart_copy_df['HadCOPD'].map(YES_NO_UNKNOWN_QUESTIONS)
heart_copy_df['HadDepressiveDisorder'] = heart_copy_df['HadDepressiveDisorder'].map(YES_NO_UNKNOWN_QUESTIONS)
heart_copy_df['HadKidneyDisease'] = heart_copy_df['HadKidneyDisease'].map(YES_NO_UNKNOWN_QUESTIONS)
heart_copy_df['HadArthritis'] = heart_copy_df['HadArthritis'].map(YES_NO_UNKNOWN_QUESTIONS)
heart_copy_df['HadDiabetes'] = heart_copy_df['HadDiabetes'].map(DIABETES)
heart_copy_df['DeafOrHardOfHearing'] = heart_copy_df['DeafOrHardOfHearing'].map(YES_NO_UNKNOWN_QUESTIONS)
heart_copy_df['BlindOrVisionDifficulty'] = heart_copy_df['BlindOrVisionDifficulty'].map(YES_NO_UNKNOWN_QUESTIONS)
heart_copy_df['DifficultyConcentrating'] = heart_copy_df['DifficultyConcentrating'].map(YES_NO_UNKNOWN_QUESTIONS)
heart_copy_df['DifficultyWalking'] = heart_copy_df['DifficultyWalking'].map(YES_NO_UNKNOWN_QUESTIONS)
heart_copy_df['DifficultyDressingBathing'] = heart_copy_df['DifficultyDressingBathing'].map(YES_NO_UNKNOWN_QUESTIONS)
heart_copy_df['DifficultyErrands'] = heart_copy_df['DifficultyErrands'].map(YES_NO_UNKNOWN_QUESTIONS)
heart_copy_df['SmokerStatus'] = heart_copy_df['SmokerStatus'].map(SMOKER_STATUS)
heart_copy_df['ECigaretteUsage'] = heart_copy_df['ECigaretteUsage'].map(ECIGARETTES)
heart_copy_df['ChestScan'] = heart_copy_df['ChestScan'].map(YES_NO_UNKNOWN_QUESTIONS)
heart_copy_df['RaceEthnicityCategory'] = heart_copy_df['RaceEthnicityCategory'].map(RACE)
heart_copy_df['AgeCategory'] = heart_copy_df['AgeCategory'].map(AGE_CATEGORY)
heart_copy_df['HeightInMeters'] = heart_copy_df['HeightInMeters'] / 100
heart_copy_df['WeightInKilograms'] = heart_copy_df['WeightInKilograms'] / 100
heart_copy_df['BMI'] = heart_copy_df['BMI'] / 100
heart_copy_df['AlcoholDrinkers'] = heart_copy_df['AlcoholDrinkers'].map(YES_NO_UNKNOWN_QUESTIONS)
heart_copy_df['FluVaxLast12'] = heart_copy_df['FluVaxLast12'].map(YES_NO_UNKNOWN_QUESTIONS)
heart_copy_df['PneumoVaxEver'] = heart_copy_df['PneumoVaxEver'].map(YES_NO_UNKNOWN_QUESTIONS)
heart_copy_df['TetanusLast10Tdap'] = heart_copy_df['TetanusLast10Tdap'].map(TETANUS)



---

## 4. Validity Check <a class="anchor" id="4"></a>

- Check the DataFrame shape are the same before and after mapping.
- Check the values in each column to examine if mapping is successful.

In [20]:
# Compare the datset shape before and after mapping
print(f"The dataset shape before mapping is {heart_df.shape}")

print(f"The dataset shape after mapping is {heart_copy_df.shape}")

The dataset shape before mapping is (433323, 36)
The dataset shape after mapping is (433323, 36)


In [21]:
# Get the value_counts of categorical columns, and summary statistics for numeric columns

for col in heart_copy_df.columns:

    # Print the column name and dtype
    col_dtype = heart_copy_df[col].dtype
    print(f"\nColumn: {col} ({col_dtype})")

    # Print the number of missing values in each column
    print(f"--- Percentage of NaNs: {(heart_copy_df[col].isna().sum() / len(heart_copy_df[col]) * 100).round(2)}")
    
    
    if col_dtype == 'object':
        # If the column is categorical, print value counts in %
        print("--- Percentage of Each Category:")
        print((heart_copy_df[col].value_counts(normalize=True) * 100).round(2))
        print('---------' * 10)
    else:
        # If the column is numeric, print summary statistics
        print(f"--- Summary statistics:\n {heart_copy_df[col].describe()}")


Column: State (object)
--- Percentage of NaNs: 0.0
--- Percentage of Each Category:
State
Washington              6.10
New York                4.00
Maryland                3.98
Minnesota               3.73
Ohio                    3.09
Florida                 3.06
Nebraska                2.97
Wisconsin               2.96
Maine                   2.83
Arizona                 2.78
California              2.76
Utah                    2.57
Indiana                 2.54
Texas                   2.32
South Carolina          2.32
Michigan                2.30
Kansas                  2.28
Massachusetts           2.20
Connecticut             2.19
New Jersey              2.15
Iowa                    2.05
Colorado                2.03
Georgia                 1.90
Hawaii                  1.81
Vermont                 1.76
Missouri                1.67
Montana                 1.65
Virginia                1.61
New Hampshire           1.61
Idaho                   1.59
Oklahoma                1.55
Oregon    

---

## 5. Data Export <a class="anchor" id="5"></a>

In [22]:
# Export the file
heart_copy_df.to_csv('../../data/Active/processed/heart_attack_with_nan_2023.csv', index=False)